In [14]:
import json
import tweepy

In [15]:
access_token = "1174895776444624896-YfvYGV7tskiD6CWg0PBm4P8ekV3pUc"
access_token_secret = "WiOBT3BZ2qLrX9owDFxBzORSjqFbKuBg8HvzX6Et7rqgl"
consumer_key = "CxmWm6ItBRDxiEmwJelzHicqn"
consumer_secret = "5aK80bORH5YsbhL35AS00LQDcTNPODuzBvC4WsAwHnMwfpWdgc"

In [16]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [17]:
api = tweepy.API(auth, parser = tweepy.parsers.JSONParser())

In [18]:
searchquery = '" " -filter:retweets'

In [19]:
data = api.search(q = searchquery, count = 100, lang = 'en', result_type = 'mixed')

In [20]:
values = list(data.values())
#print(values)

In [21]:
import time

data = api.search(q = searchquery, count = 100, lang = 'en', result_type = 'mixed')
data_all = values[0]

while (len(data_all) <= 20000):
    time.sleep(5)
    last = data_all[-1]['id']
    data = api.search(q = searchquery, count = 100, lang = 'en', result_type = 'mixed', max_id = last)
    data_all += values[0][1:]

In [26]:
tweet = []
vs_pos = []
vs_neu = []
vs_neg = []

In [27]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

for i in range(0, len(data_all)):
    tweet.append(data_all[i]['text'])
    vs_pos.append(analyzer.polarity_scores(data_all[i]['text'])['pos'])
    vs_neu.append(analyzer.polarity_scores(data_all[i]['text'])['neu'])
    vs_neg.append(analyzer.polarity_scores(data_all[i]['text'])['neg'])

In [51]:
from pandas import Series, DataFrame

twitter_df = DataFrame({'Tweet': tweet,
                        'Positive': vs_pos,
                        'Neutral': vs_neu,
                        'Negative': vs_neg})
twitter_df = twitter_df[['Tweet',
                         'Positive', 'Neutral', 'Negative']]

# Have a look at the top 5 results.
output_file = open("JSON_Sentiment_Results.txt", "w")
#print(twitter_df)
twitter_df.head()

,Tweet,Positive,Neutral,Negative
0,Let’s try it again. 💜💜💜 #MAMAVOTE #BTS @BTS_tw...,0.182,0.818,0.000
1,#WeMadeIt is out now! https://t.co/ZmjE2kvEay ...,0.000,1.000,0.000
2,Everything is MAMA. @BTS_twt #MAMAvote https:/...,0.000,1.000,0.000
3,@U2start Omg yrs pasting to quickly,0.000,1.000,0.000
4,@ChrisSalts @KHMakerD I do like Darnold. So I ...,0.265,0.618,0.116


In [48]:
twitter_df.to_csv(r'/Users/taracopas/Desktop/JSON_Sentiment_Results.csv')